# Get Mammograpy Dataset

In [ ]:
import pickle
import random
import numpy as np

def getMammographyDataset():
    file_path = '/content/dataset/dataset_256_256_CBIS-DDSM_dict.pkl'
    with open(file_path, 'rb') as file:
        dataset = pickle.load(file)

    keys = list(dataset.keys())
    random.shuffle(keys)

    split_index = int(len(keys) * 0.7)

    train_keys = keys[:split_index]
    test_keys = keys[split_index:]

    x_ray_train = np.array([dataset[key]['image_file_numpy'] for key in train_keys])
    y_roi_train = np.array([dataset[key]['roi_mask_file_numpy'] for key in train_keys])

    x_ray_test = np.array([dataset[key]['image_file_numpy'] for key in test_keys])
    y_roi_test = np.array([dataset[key]['roi_mask_file_numpy'] for key in test_keys])

    return (x_ray_train, y_roi_train), (x_ray_test, y_roi_test)

import matplotlib.pyplot as plt

(x_ray_train, y_roi_train), (x_ray_test, y_roi_test) = getMammographyDataset()
#Constantes para el tamaño de las imágenes
HEIGHT = 256
WIDTH = 256

print(x_ray_train.shape)

# Mostrar las imágenes originales y roi
n = 10  # Número de imágenes a mostrar
plt.figure(figsize=(20, 4))
for i in range(n):
    # Muestra la imagen original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_ray_test[i].reshape(HEIGHT, WIDTH), cmap="gray")
    plt.title("Original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Muestra la imagen reconstruida
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(y_roi_test[i].reshape(HEIGHT, WIDTH), cmap="gray")
    plt.title("Roi Mask")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

# Model Autoencoder - UNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def unet_autoencoder(input_shape=(256, 256, 1)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs, outputs)
    return model

# Crear el modelo
autoencoder = unet_autoencoder()

# Compilar el modelo
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Obtener el dataset
(x_ray_train, y_roi_train), (x_ray_test, y_roi_test) = getMammographyDataset()

# Verifica la forma de entrada del modelo
print(autoencoder.input_shape)

# Verifica la forma de tus datos de entrada
print(x_ray_train.shape)

In [ ]:
import tensorflow as tf

# Nombre del archivo para guardar el modelo
MODEL_SAVE_PATH = "unet_autoencoder.keras"

# Entrenar el modelo
autoencoder.fit(x_ray_train, y_roi_train, epochs=35, batch_size=20, validation_data=(x_ray_test, y_roi_test))

#Guardar el modelo entrenado
autoencoder.save(MODEL_SAVE_PATH)

In [ ]:
## CARGA DEL MODELO
import tensorflow as tf
from tensorflow import keras

# Carga el modelo desde el archivo .keras
autoencoder = keras.models.load_model("unet_autoencoder.keras")

In [ ]:
# Generar una imagen a partir de una entrada
decoded_imgs = autoencoder.predict(x_ray_test)
predictions_bin = (decoded_imgs > 0.5).astype(np.uint8)

In [ ]:
import matplotlib.pyplot as plt

HEIGHT = 256
WIDTH = 256

# Mostrar las imágenes originales y generadas
n = 15  # Número de imágenes a mostrar
plt.figure(figsize=(20, 4))
for i in range(n):
    # Muestra la imagen original
    ax = plt.subplot(3, n, i + 1)
    plt.imshow(x_ray_test[i].reshape(HEIGHT, WIDTH), cmap="gray")
    plt.title("Original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Muestra la imagen reconstruida
    ax = plt.subplot(3, n, i + 1 + n)
    plt.imshow(predictions_bin[i].reshape(HEIGHT, WIDTH), cmap="gray")
    plt.title("Generada")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Muestra la imagen original del roi
    ax = plt.subplot(3, n, i + 1 + n + n)
    plt.imshow(y_roi_test[i].reshape(HEIGHT, WIDTH), cmap="gray")
    plt.title("Original Roi")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

In [ ]:
import numpy as np

def dice_coefficient(y_true, y_pred):
    """
    Calcula el coeficiente de Dice entre dos máscaras binarias.

    Parámetros:
    - y_true: np.ndarray, máscara de referencia (ground truth).
    - y_pred: np.ndarray, máscara predicha.

    Retorna:
    - coeficiente de Dice (float).
    """
    y_true = np.asarray(y_true).astype(bool)
    y_pred = np.asarray(y_pred).astype(bool)

    if y_true.shape != y_pred.shape:
        raise ValueError("Las máscaras deben tener la misma forma.")

    intersection = np.logical_and(y_true, y_pred).sum()
    total = y_true.sum() + y_pred.sum()

    if total == 0:
      print('vacias')
      return 1.0  # Ambas máscaras están vacías

    return 2.0 * intersection / total

In [ ]:
dice_scores = []

for i in range(len(x_ray_test)):
    y_true = y_roi_test[i].squeeze()
    y_pred = predictions_bin[i].squeeze()

    dice = dice_coefficient(y_true, y_pred)

    dice_scores.append(dice)
    
max(dice_scores)

In [ ]:
import matplotlib.pyplot as plt

eje_x = list(range(1, 199))

# Datos de ejemplo
x = eje_x
y = dice_scores

# Crear la figura y los ejes
fig, ax = plt.subplots()

# Trazar los datos
ax.plot(x, y, marker='o')  # 'o' define un marcador circular en cada punto

# Personalizar el gráfico
ax.set_xlabel("Predictions vs Test images")
ax.set_ylabel("Dice Coefficient")
ax.set_title("Autoencoder-Unet")

# Mostrar el gráfico
plt.show()